In [1]:
import os, sys, subprocess
import pandas as pd
df = pd.read_csv('DustPedia_Aperture_Photometry_2.2.csv')

In [2]:
# Rearrange in order of increasing effective wavelenght
right_order = [u'name', u'ra', u'dec', u'semimaj_arcsec', u'axial_ratio', u'pos_angle', u'global_flag',
    u'GALEX_FUV', u'GALEX_FUV_err', u'GALEX_FUV_flag', u'GALEX_NUV', u'GALEX_NUV_err', u'GALEX_NUV_flag',
    u'SDSS_u', u'SDSS_u_err', u'SDSS_u_flag', u'SDSS_g', u'SDSS_g_err', u'SDSS_g_flag',
    u'SDSS_r', u'SDSS_r_err', u'SDSS_r_flag', u'SDSS_i', u'SDSS_i_err', u'SDSS_i_flag', u'SDSS_z', u'SDSS_z_err', u'SDSS_z_flag',
    u'2MASS_J', u'2MASS_J_err', u'2MASS_J_flag', u'2MASS_H', u'2MASS_H_err', u'2MASS_H_flag', u'2MASS_Ks', u'2MASS_Ks_err', u'2MASS_Ks_flag',
    u'WISE_3.4', u'WISE_3.4_err', u'WISE_3.4_flag', u'Spitzer_3.6', u'Spitzer_3.6_err', u'Spitzer_3.6_flag',
    u'Spitzer_4.5', u'Spitzer_4.5_err', u'Spitzer_4.5_flag', u'WISE_4.6', u'WISE_4.6_err', u'WISE_4.6_flag',
    u'Spitzer_5.8', u'Spitzer_5.8_err', u'Spitzer_5.8_flag', u'Spitzer_8.0', u'Spitzer_8.0_err', u'Spitzer_8.0_flag',
    u'WISE_12', u'WISE_12_err', u'WISE_12_flag', u'WISE_22', u'WISE_22_err', u'WISE_22_flag',
    u'Spitzer_24', u'Spitzer_24_err', u'Spitzer_24_flag', u'Spitzer_70', u'Spitzer_70_err', u'Spitzer_70_flag',
    u'PACS_70', u'PACS_70_err', u'PACS_70_flag', u'PACS_100', u'PACS_100_err', u'PACS_100_flag',
    u'PACS_160', u'PACS_160_err', u'PACS_160_flag', u'Spitzer_160', u'Spitzer_160_err', u'Spitzer_160_flag',
    u'SPIRE_250', u'SPIRE_250_err', u'SPIRE_250_flag', u'SPIRE_350', u'SPIRE_350_err', u'SPIRE_350_flag',
    u'SPIRE_500', u'SPIRE_500_err', u'SPIRE_500_flag']
df = df[right_order]

In [19]:
gal_phot = df.loc[df['name'] == 'NGC1068']
# Fist, remove _flag columns
to_remove = gal_phot.columns.str.contains('flag', case=False)
gal_phot = gal_phot.loc[:,~to_remove]
# Extract ra, dec, semimaj, axial ratio and pos_angle, then remove them
ra, dec = gal_phot['ra'].values[0], gal_phot['dec'].values[0]
semimaj, axial_ratio, pos_angle = gal_phot['semimaj_arcsec'].values[0], gal_phot['axial_ratio'].values[0], gal_phot['pos_angle'].values[0]
to_remove = ['name', 'ra', 'dec', 'semimaj_arcsec', 'axial_ratio', 'pos_angle']
gal_phot = gal_phot.drop(columns=to_remove)
# And remove empy columns
gal_phot = gal_phot.dropna(axis='columns')

In [70]:
# Extract working bands
working_bands = ['GALEX_FUV', 'GALEX_NUV', 'SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z', \
                 '2MASS_J', '2MASS_H', '2MASS_Ks', 'WISE_3.4', 'Spitzer_3.6', 'Spitzer_4.5', \
                 'WISE_4.6', 'Spitzer_5.8', 'Spitzer_8.0', 'WISE_12', 'WISE_22', 'Spitzer_24',\
                 'PACS_70', 'PACS_100', 'PACS_160', 'SPIRE_250', 'SPIRE_350']

working_bands_err = [t+'_err' for t in working_bands]
gal_phot_flux = gal_phot[working_bands]
gal_phot_flux = gal_phot_flux.transpose()
gal_phot_err = gal_phot[working_bands_err]
gal_phot_err = gal_phot_err.transpose()
galaxy_photometry = pd.DataFrame(np.concatenate((gal_phot_flux.values, gal_phot_err.values), axis=1))
galaxy_photometry.columns = ['Flux', 'Error']
galaxy_photometry.index = working_bands
galaxy_photometry.to_csv('')

In [10]:
galaxies = ['NGC0628', 'NGC3184', 'NGC3938', 'NGC4254', 'NGC4321', 'NGC4535', 'NGC5194', 'NGC5457', 'NGC7331', 'NGC5055']
list_z = [0.0020336, 0.0025081, 0.0049484, 0.0031408, 0.0031408, 0.0036379, 0.0017851, 0.0015817, 0.00314079, 0.00198233] # FAKE z, from D[Mpc]
z_dictionary = dict(zip(galaxies, list_z))

for galaxy_name in galaxies:
    #if os.path.exists('../'+galaxy_name+'/magphys_integrato/AUTO_observations_integ.dat'): continue
    gal_phot = df.loc[df['name'] == galaxy_name]
    print galaxy_name
    print gal_phot['ra'], gal_phot['dec'], gal_phot['semimaj_arcsec'], gal_phot['axial_ratio'], gal_phot['pos_angle']
    # Fist, remove _flag columns
    to_remove = gal_phot.columns.str.contains('flag', case=False)
    gal_phot = gal_phot.loc[:,~to_remove]
    # Remove name, ra, dec, semimaj, axial ratio, pos angle columns
    to_remove = ['name', 'ra', 'dec', 'semimaj_arcsec', 'axial_ratio', 'pos_angle']
    gal_phot = gal_phot.drop(columns=to_remove)
    # Remove SPIRE500, Spitzer 160, Spitzer 70
    to_remove = ['Spitzer_160', 'Spitzer_160_err', 'Spitzer_70', 'Spitzer_70_err']
    gal_phot = gal_phot.drop(columns=to_remove)
    # And remove empy columns
    gal_phot = gal_phot.dropna(axis='columns')
    # Add z and id
    gal_phot['z'] = z_dictionary[galaxy_name]
    gal_phot = gal_phot[['z'] + gal_phot.columns[:-1].tolist()]
    gal_phot['id'] = 'Int'+galaxy_name[3:]
    gal_phot = gal_phot[['id'] + gal_phot.columns[:-1].tolist()]
    # Save and move to observations_integ.dat correct folder
    gal_phot.to_csv(galaxy_name+'_photometry.dat', sep='\t', index = False)
    #try:
    #    subprocess.call('cp -rp '+galaxy_name+'_photometry.csv /home/enia/magphys/'+galaxy_name+'/run_integrated/AUTO_observations_integ.dat', shell = True)
    #    os.rename(galaxy_name+'_photometry.csv', '../'+galaxy_name+'/magphys_integrato/AUTO_observations_integ.dat')
    #except:
    #    subprocess.call('mkdir ../'+galaxy_name+'/magphys_integrato', shell = True)
    #    subprocess.call('mkdir /home/enia/magphys/'+galaxy_name+'/run_integrated/', shell = True)
    #    os.rename(galaxy_name+'_photometry.csv', '../'+galaxy_name+'/magphys_integrato/AUTO_observations_integ.dat')

NGC0628
39    24.174
Name: ra, dtype: float64 39    15.78332
Name: dec, dtype: float64 39    586.89212
Name: semimaj_arcsec, dtype: float64 39    1.020941
Name: axial_ratio, dtype: float64 39   -3.133546
Name: pos_angle, dtype: float64
NGC3184
207    154.5708
Name: ra, dtype: float64 207    41.42436
Name: dec, dtype: float64 207    319.301926
Name: semimaj_arcsec, dtype: float64 207    1.040651
Name: axial_ratio, dtype: float64 207    11.227655
Name: pos_angle, dtype: float64
NGC3938
321    178.2057
Name: ra, dtype: float64 321    44.1208
Name: dec, dtype: float64 321    271.731517
Name: semimaj_arcsec, dtype: float64 321    1.129579
Name: axial_ratio, dtype: float64 321    130.406092
Name: pos_angle, dtype: float64
NGC4254
414    184.70655
Name: ra, dtype: float64 414    14.41641
Name: dec, dtype: float64 414    421.336905
Name: semimaj_arcsec, dtype: float64 414    1.347099
Name: axial_ratio, dtype: float64 414   -35.74497
Name: pos_angle, dtype: float64
NGC4321
459    185.7282
Name:

In [4]:
galaxies = ['NGC0628', 'NGC3184', 'NGC3938', 'NGC4254', 'NGC4321', 'NGC4535', 'NGC5194', 'NGC5457']

list_wvl = [1.52e-01, 2.27e-01,
           3.55e-01, 4.68e-01, 6.16e-01, 7.48e-01, 8.93e-01,
           1.25e+00, 1.65e+00, 2.16e+00,
           3.40e+00, 3.60e+00, 4.50e+00, 4.60e00, 5.80e+00, 8.00e+00, 1.20e+01, 2.20e+01, 2.40e+01, 
           7.00e+01, 1.00e+02, 1.60e+02, 2.50e+02, 3.50e+02]
list_band =  ['GALEX_FUV', 'GALEX_NUV',
              'SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z',
              '2MASS_J', '2MASS_H', '2MASS_Ks',
              'WISE_3.4', 'Spitzer_3.6', 'Spitzer_4.5', 'WISE_4.6', 'Spitzer_5.8', 'Spitzer_8.0', 'WISE_12', 'WISE_22', 'Spitzer_24',
              'PACS_70', 'PACS_100', 'PACS_160', 'SPIRE_250', 'SPIRE_350']

band_wvl_dictionary = dict(zip(list_band, list_wvl))

for galaxy_name in galaxies:
    if os.path.exists('../'+galaxy_name+'/MyReduction/AUTO_Dustpedia_Photometry.csv'): continue
    gal_phot = df.loc[df['name'] == galaxy_name]
    gal_phot.reset_index(drop=True)
    # Remove _flag columns
    to_remove = gal_phot.columns.str.contains('flag', case=False)
    gal_phot = gal_phot.loc[:,~to_remove]
    # Remove _error columns
    to_remove = gal_phot.columns.str.contains('err', case=False)
    gal_phot = gal_phot.loc[:,~to_remove]
    # Remove name, ra, dec, semimaj, axial ratio, pos angle columns
    to_remove = ['name', 'ra', 'dec', 'semimaj_arcsec', 'axial_ratio', 'pos_angle']
    gal_phot = gal_phot.drop(columns=to_remove)
    # Remove SPIRE500 and Spitzer 160
    to_remove = ['SPIRE_500', 'Spitzer_160', 'Spitzer_70']
    gal_phot = gal_phot.drop(columns=to_remove)
    # And remove empy columns
    gal_phot = gal_phot.dropna(axis='columns')
    # Add wavelengths
    lw = []
    for band in gal_phot.columns.values:
        lw.append(band_wvl_dictionary[band])
    gal_phot = gal_phot.T
    gal_phot.rename(columns={gal_phot.columns[0]:'flux'}, inplace=True)    
    gal_phot.columns.name = 'Bands'
    gal_phot['wvl'] = lw  
    gal_phot = gal_phot[['wvl'] + gal_phot.columns[:-1].tolist()]
    # Save
    gal_phot.to_csv(galaxy_name+'_photometry.csv', sep='\t')
    
    #try:
    #    os.rename(galaxy_name+'_photometry.csv', '../'+galaxy_name+'/MyReduction/AUTO_Dustpedia_Photometry.csv')
    #except:
    #    subprocess.call('mkdir ../'+galaxy_name+'/MyReduction', shell = True)
    #    os.rename(galaxy_name+'_photometry.csv', '../'+galaxy_name+'/MyReduction/AUTO_Dustpedia_Photometry.csv')